In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pandas.api.types import is_numeric_dtype
from sklearn.ensemble import BaggingClassifier, StackingClassifier
from sklearn.metrics import classification_report

from knowledge_distillation.io import *
from knowledge_distillation.ensemble import *
from knowledge_distillation.processing import * 

In [ ]:
import numpy as np
import pandas as pd
# from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# from category_encoders import OneHotEncoder, OrdinalEncoder
# from sklearn_pandas import DataFrameMapper, gen_features


# Parameters

In [ ]:
max_iter = 100

# Load data

In [ ]:
df = load_adult()
assert df.shape == (32561, 15)

In [ ]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
df.salary.value_counts(dropna=False)

 <=50K    24720
 >50K      7841
Name: salary, dtype: int64

# Preprocessing

In [ ]:
X, y, target_names = scale_onehot(df, target='salary')

X_train, X_test, y_train, y_test = split_with_seed(X, y)

/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif

# Train a Logistic Regression

In [ ]:
lr = LogisticRegression(max_iter=max_iter, random_state=1, n_jobs=-1)

In [ ]:
lr.fit(X_train, y=y_train)

LogisticRegression(n_jobs=-1, random_state=1)

# Evaluate single model

In [ ]:
evaluate_model(X_train, X_test, y_train, y_test, lr, 'lr', save_to_disk=True, target_names=target_names)

=== Train ===
              precision    recall  f1-score   support

       <=50K       0.88      0.93      0.91     19778
        >50K       0.74      0.60      0.66      6270

    accuracy                           0.85     26048
   macro avg       0.81      0.77      0.78     26048
weighted avg       0.85      0.85      0.85     26048


=== Test ===
              precision    recall  f1-score   support

       <=50K       0.88      0.93      0.91      4942
        >50K       0.75      0.61      0.68      1571

    accuracy                           0.86      6513
   macro avg       0.82      0.77      0.79      6513
weighted avg       0.85      0.86      0.85      6513




,model_name,data,accuracy,precision,recall,f1,auc
0,lr,train,0.852158,0.735266,0.602871,0.662519,0.767028
1,lr,test,0.857669,0.750000,0.614895,0.675761,0.774870


# Bagging of Logistic Regression models

In [ ]:
from sklearn.ensemble import BaggingClassifier

bag = BaggingClassifier(
    base_estimator=LogisticRegression(max_iter=max_iter),
    max_samples=.7,
    max_features=.7,
    bootstrap=True,
    bootstrap_features=True,
    n_jobs=-1
#     random_state=1
)

In [ ]:
bag.fit(X_train, y_train)

BaggingClassifier(base_estimator=LogisticRegression(), bootstrap_features=True,
                  max_features=0.7, max_samples=0.7, n_jobs=-1)

In [ ]:
evaluate_model(X_train, X_test, y_train, y_test, bag, 'lr_bag', save_to_disk=False, target_names=target_names)

=== Train ===
              precision    recall  f1-score   support

       <=50K       0.86      0.95      0.90     19778
        >50K       0.76      0.52      0.62      6270

    accuracy                           0.85     26048
   macro avg       0.81      0.74      0.76     26048
weighted avg       0.84      0.85      0.84     26048


=== Test ===
              precision    recall  f1-score   support

       <=50K       0.87      0.95      0.91      4942
        >50K       0.78      0.54      0.63      1571

    accuracy                           0.85      6513
   macro avg       0.82      0.74      0.77      6513
weighted avg       0.84      0.85      0.84      6513




,model_name,data,accuracy,precision,recall,f1,auc
0,lr_bag,train,0.846322,0.764898,0.522010,0.620533,0.735572
1,lr_bag,test,0.850760,0.776547,0.535328,0.633760,0.743180


In [ ]:
lr.predict(X_train)

array([0, 1, 0, ..., 0, 0, 0], dtype=int8)

# Ensemble identical models with different random seed and average

In [ ]:
# LR with random state 1..10
estimators = [(f'lr_{x}', LogisticRegression(max_iter=max_iter, random_state=x)) for x in range(10)]

stack = StackingClassifier(
    estimators, 
    final_estimator=UnbiasedAverage(), 
    n_jobs=-1)

stack.fit(X_train, y_train)

StackingClassifier(estimators=[('lr_0', LogisticRegression(random_state=0)),
                               ('lr_1', LogisticRegression(random_state=1)),
                               ('lr_2', LogisticRegression(random_state=2)),
                               ('lr_3', LogisticRegression(random_state=3)),
                               ('lr_4', LogisticRegression(random_state=4)),
                               ('lr_5', LogisticRegression(random_state=5)),
                               ('lr_6', LogisticRegression(random_state=6)),
                               ('lr_7', LogisticRegression(random_state=7)),
                               ('lr_8', LogisticRegression(random_state=8)),
                               ('lr_9', LogisticRegression(random_state=9))],
                   final_estimator=UnbiasedAverage(), n_jobs=-1)

In [ ]:
evaluate_model(X_train, X_test, y_train, y_test, stack, 'lr_ensemble', save_to_disk=True, target_names=target_names)

=== Train ===
              precision    recall  f1-score   support

       <=50K       0.88      0.93      0.91     19778
        >50K       0.74      0.60      0.66      6270

    accuracy                           0.85     26048
   macro avg       0.81      0.77      0.78     26048
weighted avg       0.85      0.85      0.85     26048


=== Test ===
              precision    recall  f1-score   support

       <=50K       0.88      0.93      0.91      4942
        >50K       0.75      0.61      0.68      1571

    accuracy                           0.86      6513
   macro avg       0.82      0.77      0.79      6513
weighted avg       0.85      0.86      0.85      6513




,model_name,data,accuracy,precision,recall,f1,auc
0,lr_ensemble,train,0.852158,0.735266,0.602871,0.662519,0.767028
1,lr_ensemble,test,0.857669,0.750000,0.614895,0.675761,0.774870


In [ ]:
for name, clf in stack.named_estimators_.items():
    evaluate_model(X_train, X_test, y_train, y_test, clf, name, save_to_disk=True, print_report=False, target_names=target_names)